In [ ]:
from gboml import GbomlGraph

gboml_model = GbomlGraph(1*24*365)
nodes, edges, _ = gboml_model.import_all_nodes_and_edges("../models/benchmark_mitSpeicher.txt")
gboml_model.add_nodes_in_model(*nodes)
gboml_model.add_hyperedges_in_model(*edges)
gboml_model.build_model()
solution = gboml_model.solve_gurobi(details=True)
(solution_flat, objective, status, solver_info, constr_info, var_info) = solution
solution_dict = gboml_model.turn_solution_to_dictionary(solution=solution_flat, solver_data=solver_info, status=status, objective=objective, constraint_info=constr_info, variables_info=var_info)
print(solution)

In [ ]:
import json
#with open('../results/result.json', 'w') as res:
#    json.dump(solution_dict["solution"], res)

In [ ]:
solution_dict["solution"]["elements"].keys()

In [ ]:
solution_dict["solution"]["elements"]["BATTERY"]["variables"].keys()

In [ ]:
solution_dict["solution"]["elements"]["BATTERY"]["variables"].keys()

In [ ]:

solution_dict["solution"]["elements"]["BATTERY"]["variables"]["discharge"]

In [ ]:
import numpy as np
discharge = np.array(solution_dict["solution"]["elements"]["BATTERY"]["variables"]["discharge"]["values"])

In [ ]:
import matplotlib.pyplot as plt

plt.plot(discharge[-24:])
plt.show()

In [ ]:
soc = np.array(solution_dict["solution"]["elements"]["BATTERY"]["variables"]["soc"]["values"])
plt.plot(soc[:24])
plt.show()